# $\text{sinc}(x)$ and Its Derivatives
\begin{align}
\text{sinc}(x) &= \begin{cases} 
      \frac{x^4}{120}-\frac{x^2}{6}+1 & |x| \leq \epsilon \\
      \frac{\sin(x)}{x} & \text{otherwise}
   \end{cases}\\
\end{align}

In [1]:
import sys

import numpy
from sympy import *
from sympy.plotting import plot

x = Symbol("x")
x0 = Symbol("x_0")

In [2]:
def taylor_series(f, n=5):
    dfs = [f]
    for i in range(n):
        dfs.append(dfs[i].diff())
    return sum([
        dfs[i].subs({"x": x0}) / factorial(i) * (x - x0)**i
        for i in range(len(dfs))
    ])

In [3]:
sinc = sin(x) / x
display(sinc)

sin(x)/x

In [4]:
sinc_ts = taylor_series(sinc)
display(limit(sinc_ts, x0, 0))

x**4/120 - x**2/6 + 1

In [5]:
x1 = 1e-3
print(1.0 - x1**2 / 6)
print(1.0 - x1**2 * (1 / 6 + x1**2 / 120))
x1 = sys.float_info.epsilon
print(N(x1, 50))
print(f"{N(sqrt(x1), 35):e}")
print(f"{N(sqrt(sqrt(x1)), 25):e}")

0.9999998333333333
0.999999833333325
2.2204460492503130808472633361816406250000000000000e-16
1.4901161193847656250000000000000000e-8
1.220703125000000000000000e-4


## $\frac{d}{dx} \text{sinc}(x)$
\begin{align}
\text{sinc}'(x) &= \begin{cases} 
      0 & x = 0 \\
      \frac{x\cos(x) - \sin(x)}{x^2} & \text{otherwise}
   \end{cases}\\
\end{align}

In [6]:
dsinc = simplify(sinc.diff(x))
display(dsinc)

(x*cos(x) - sin(x))/x**2

In [7]:
dsinc_ts = taylor_series(dsinc)
display(limit(dsinc_ts, x0, 0))

-x**5/840 + x**3/30 - x/3

## $\frac{d^2}{dx^2} \text{sinc}(x)$
\begin{align}
\text{sinc}''(x) &= \begin{cases} 
      \frac{-1}{3} & x = 0 \\
      \frac{-x^2\sin(x) - 2x\cos(x) + 2\sin(x)}{x^3} & \text{otherwise}
   \end{cases}\\
\end{align}

In [8]:
ddsinc = simplify(dsinc.diff(x))
display(ddsinc)

(-x**2*sin(x) - 2*x*cos(x) + 2*sin(x))/x**3

In [9]:
ddsinc_ts = taylor_series(ddsinc)
display(limit(ddsinc_ts, x0, 0))

-x**4/168 + x**2/10 - 1/3

## Derivatives of $sinc(\|x\|)$

In [10]:
def norm(x):
    return sqrt(sum(x**2))

In [11]:
X = numpy.array(symbols("x y z"))
sinc_normx = sinc.subs({"x": norm(X)})
display(sinc_normx)

sin(sqrt(x**2 + y**2 + z**2))/sqrt(x**2 + y**2 + z**2)

## Gradient of $\text{sinc}(\|x\|)$
\begin{align}
\nabla \text{sinc}(\|x\|) &= \frac{\text{sinc}'(\|x\|)}{\|x\|}x \\
\frac{\text{sinc}'(x)}{x} &= \begin{cases}
      -\frac{x^4}{840} + \frac{x^2}{30} - \frac{1}{3}& |x| \leq \epsilon\\
      \frac{x\cos(x)-\sin(x)}{x^3} & \text{otherwise}
   \end{cases}
\end{align}
where $\epsilon = 10^{-4}$.

In [12]:
dsinc_normx = numpy.array(sinc_normx.diff(X))
my_dsinc_normx = dsinc.subs({"x": norm(X)}) / norm(X) * X
for i in range(3):
    assert (simplify((dsinc_normx - my_dsinc_normx)[i]) == 0)

In [13]:
dsinc_x = dsinc / x
display(dsinc_x)

(x*cos(x) - sin(x))/x**3

In [14]:
dsinc_x_ts = taylor_series(dsinc_x)
display(limit(dsinc_x_ts, x0, 0))

-x**4/840 + x**2/30 - 1/3

## Hessian of $\text{sinc}(\|x\|)$
\begin{align}
\nabla^2 \text{sinc}(\|x\|) &= \left(\frac{\text{sinc}''(\|x\|)}{\|x\|^2} - \frac{\text{sinc}'(\|x\|)}{\|x\|^3}\right)xx^T + \frac{\text{sinc}'(\|x\|)}{\|x\|}I \\
\frac{\text{sinc}''(\|x\|)}{\|x\|^2} - \frac{\text{sinc}'(\|x\|)}{\|x\|^3} &= \begin{cases} 
      \frac{x^{4}}{7560}-\frac{x^{2}}{210}+\frac{1}{15} & |x| \leq \epsilon \\
      \frac{-x^{2} \sin (x)-3 x \cos (x)+3 \sin (x)}{x^{5}} & \text{otherwise}
   \end{cases}
\end{align}
where $\epsilon = 0.1$.

In [15]:
ddsinc_normx = numpy.array(Matrix(Matrix(dsinc_normx).diff(X)))
my_ddsinc_normx = ((ddsinc.subs({"x": norm(X)}) / norm(X)**2 -
                    dsinc.subs({"x": norm(X)}) / norm(X)**3) *
                   X.reshape(3, 1) @ X.reshape(1, 3) +
                   dsinc.subs({"x": norm(X)}) / norm(X) * numpy.eye(3))
for i in range(3):
    for j in range(3):
        assert (simplify(ddsinc_normx[i, j][0] - my_ddsinc_normx[i, j]) == 0)

In [16]:
ddsinc_x = simplify(ddsinc.subs({"x": x}) / x**2 - dsinc.subs({"x": x}) / x**3)
display(ddsinc_x)

(-x**2*sin(x) - 3*x*cos(x) + 3*sin(x))/x**5

In [17]:
ddsinc_x_ts = taylor_series(ddsinc_x)
display(limit(ddsinc_x_ts, x0, 0.0))

x**4/7560 - x**2/210 + 1/15